In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, f1_score, hamming_loss
from sklearn.preprocessing import MultiLabelBinarizer
import torch
from transformers import DistilBertTokenizer, AutoTokenizer
from transformers import DistilBertForSequenceClassification, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from transformers import EvalPrediction
import pickle

In [ ]:
data = pd.read_csv('./IMDB-Movie-Data.csv', usecols = ["Genre", "Description"])

data.head()

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data.duplicated().sum()

In [ ]:
data['Description'].str.len().plot.hist(bins=50)

In [ ]:
data['Genre'] = data['Genre'].str.split(',')

data.head()

In [ ]:
genres = [g for genre in data['Genre'] for g in genre]

genres

In [ ]:
genre_counts = pd.Series(genres).value_counts()

genre_counts

### Label Encoder

In [ ]:
multilabel = MultiLabelBinarizer()

In [ ]:
labels = multilabel.fit_transform(data['Genre']).astype('float32')

labels

In [ ]:
texts = data['Description'].tolist()

texts

### Model Building

In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size = 0.2,
                                                                      random_state = 42)

In [ ]:
model_ckpt = 'distilbert-base-uncased'
tokenizer = DistilBertTokenizer.from_pretrained(model_ckpt)
model = DistilBertForSequenceClassification.from_pretrained(model_ckpt, num_labels = len(labels[0]),
                                            problem_type = 'multi_label_classification')

In [ ]:
# Building Custom Dataset
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len = 128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)
        
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = torch.tensor(self.labels[idx])

        encoding = self.tokenizer(text, truncation = True, padding = "max_length", max_length = self.max_len, return_tensors = 'pt')

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': label
        }

In [ ]:
train_dataset = CustomDataset(train_texts, train_labels, tokenizer)
val_dataset = CustomDataset(val_texts, val_labels, tokenizer)

In [ ]:
train_dataset[0]

In [ ]:
val_dataset[0]

### Multi-Label Classification Evaluation Metrics

In [ ]:
def multi_labels_metrics(predictions, labels, threshold=0.3):
  sigmoid = torch.nn.Sigmoid()
  probs = sigmoid(torch.Tensor(predictions))

  y_pred = np.zeros(probs.shape)
  y_pred[np.where(probs>=threshold)] = 1
  y_true = labels

  f1 = f1_score(y_true, y_pred, average = 'macro')
  roc_auc = roc_auc_score(y_true, y_pred, average = 'macro')
  hamming = hamming_loss(y_true, y_pred)

  metrics = {
      "roc_auc": roc_auc,
      "hamming_loss": hamming,
      "f1": f1
  }

  return metrics

def compute_metrics(p:EvalPrediction):
  preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions

  result = multi_labels_metrics(predictions=preds,
                                labels=p.label_ids)

  return result

In [ ]:
args = TrainingArguments(
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    output_dir = './results',
    num_train_epochs=5,
    save_steps=1000,
    save_total_limit=2
)

trainer = Trainer(model=model,
                  args=args,
                  train_dataset=train_dataset,
                  eval_dataset = val_dataset,
                  compute_metrics=compute_metrics)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
trainer.save_model('./distilbert_finetuned_multilabel')

In [ ]:
with open("multi-label_binarizer.pkl", "wb") as f:
    pickle.dump(multilabel, f)

### Predictions

In [ ]:
text = "Low-level FBI agent Peter Sutherland works in the basement of the White House manning a phone that never rings - until the night it does, propelling him into a conspiracy that leads all the way to the Oval Office"

encoding = tokenizer(text, return_tensors = 'pt')
encoding

In [ ]:
encoding.to(trainer.model.device)
outputs = trainer.model(**encoding)

outputs

In [ ]:
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(outputs.logits[0].cpu())

probs

In [ ]:
preds = np.zeros(probs.shape)

preds

In [ ]:
preds[np.where(probs >= 0.3)] = 1

preds

In [ ]:
multilabel.classes_

In [ ]:
multilabel.inverse_transform(preds.reshape(1, -1))